In [1]:
using OMJulia, Plots, CSV, DataFrames

In [2]:
# OpenModelicaファイルのPATHの指定
model_name = "C:/work/test/BottleCooling1DCAE/CoolingBottle.mo"
CoolingBottle = OMJulia.OMCSession() 
CoolingBottle.ModelicaSystem(model_name,"CoolingBottle")


In [6]:
# CSVファイルをDataFrameとして読み込む
params = CSV.read("CoolingBottleParameters.csv", DataFrame)

,条件名,初期ミルク温度 [K],初期ボトル温度 [K],冷却水温度 [K],熱伝達係数*面積 [W/K]
,String,Float64,Float64,Float64,Float64
1,ガラス哺乳瓶,363.15,363.15,288.15,67.4
2,プラスチック哺乳瓶,363.15,363.15,288.15,17.9


In [7]:
# 条件数を取得
num_rows = nrow(params)

# 初期値リスト
# 条件ごとのミルクの初期温度のリスト
t0_milk_list = [params[i, :2] for i in 1:num_rows]
# 条件ごとの哺乳瓶の初期温度のリスト
t0_bottle_list = [params[i, :3] for i in 1:num_rows]
# 条件ごとの冷却水温度のリスト
t_water_list = [params[i, :4] for i in 1:num_rows]

# モデルパラメータリスト
# 条件ごとの熱伝達係数[W/m2・K]×伝熱面積[m2]の値のリスト
h_interface_list = [params[i, :5] for i in 1:num_rows]
# 条件ごとの哺乳瓶の熱コンダクタンス[W/K]のリスト
g_bottle_list = [params[i, :6] for i in 1:num_rows]
# 条件ごとのミルクの熱容量[J/K]のリスト
hc_milk_list = [params[i, :7] for i in 1:num_rows]
# 条件ごとの哺乳瓶の熱容量[J/K]のリスト
hc_bottle_list = [params[i, :8] for i in 1:num_rows]

2-element Vector{Float64}:
 75.4
 75.4

In [8]:
# 計算時間の設定
# 開始時間
time_start = 0.0 # [s]
# 終了時間
time_stop = 10.0 * 60 # [s]

# 計算時の時間刻み
dt = 1.0 # [s]
step_size = dt

# 開始時間、終了時間、時間刻みの設定
CoolingBottle.setSimulationOptions([
    "startTime=$time_start",
    "stopTime=$time_stop",
    "stepSize=$step_size"
])


In [9]:
# 結果取得対象のパラメータリストを定義
output_labels = [
    "time",
    "T_milk",
    "T_bottle",
]

# 空のDataFrameを作成
df = DataFrame()

for i in 1:num_rows
    # 初期値をリストから取得
    # ミルクの初期温度を代入
    t0_milk = t0_milk_list[i]
    # 哺乳瓶の初期温度を代入
    t0_bottle = t0_bottle_list[i]
    # 冷却水温度を代入
    t_water = t_water_list[i]
    
    # モデルパラメータをリストから取得
    # 熱伝達係数[W/m2・K]×伝熱面積[m2]の値を代入
    h_interface = h_interface_list[i]
    # 哺乳瓶の熱コンダクタンス[W/K]を代入
    g_bottle = g_bottle_list[i]
    # ミルクの熱容量[J/K]を代入
    hc_milk = hc_milk_list[i]
    # 哺乳瓶の熱容量[J/K]を代入
    hc_bottle = hc_bottle_list[i]

    # Modelicaの入力パラメータに定義した変数を代入
    CoolingBottle.setParameters([
        "T0Milk=$t0_milk",
        "T0Bottle=$t0_bottle",
        "TWater=$t_water",
        "HInterface=$h_interface",
        "GBottle=$g_bottle",
        "HCMilk=$hc_milk",
        "HCBottle=$hc_bottle",
    ])
    
    # ModelicaファイルのSimulation実行
    CoolingBottle.simulate()
    
    # 結果取得
    time,t_milk,t_bottle = CoolingBottle.getSolutions(output_labels)
    # 要素数を取得
    rows = size(time)[1]
    # 温度単位[K]を[℃に]変換する処理
    conversion_list = [-273.15 for t in 1:rows]
    t_milk_c = t_milk + conversion_list
    t_bottle_c = t_bottle + conversion_list

    # 1条件目の結果をDataFrameに格納
    if i < 2
        df.Time = time
        df[:,2] = t_milk_c
        df[:,3] = t_bottle_c

    # 2条件目の結果をDataFrameに格納
    # @TODO 3条件目以降の条件があるときに汎用性を持たせるようにしたい
    else
        df[:,4] = t_milk_c
        df[:,5] = t_bottle_c
    end

end
# グラフ描画
plot(df.Time, [df[:,2], df[:,3], df[:4], df[:5]], label=["glass_milk" "glass_bottle" "plastic_milk" "plastic_bottle"])
xlabel!("Time [s]")
ylabel!("Temperature [℃]")
title!("CoolingBottle")
# グラフ画像をpng形式保存
savefig("CoolingBottleJulia.png")